## 10分类 10个神经元 ResNet
test 准确率：64%, fine tune: 数据增强+深层+批归一化之后会达到94%

In [1]:
import tensorflow as tf

#### 如果需要使用tf 1.0
tf v2.0 does not have placeholder(), .reset_default_graph().

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#读数据：
import os
import pickle as cPickle
import numpy as np

CIFAR_DIR = "../../../other_datasets/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', '.DS_Store', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [4]:
#将数据读取,返回data 和 labels
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'latin1')
        return data['data'], data['labels']

### 将 残差连接块 抽象生成函数：

In [5]:
def residual_block(x, output_channel, is_training):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel: #将进行 对原始输入进行降采样（步长==2）并且扩充输出通道数. 因为我们设置的，想要输出的通道数，大于我们实际的输入通道数
        increase_dim = True
        strides = (2,2)
    elif input_channel == output_channel: #将不进行 对原始输入进行降采样
        increase_dim = False
        strides = (1,1)
    else:
        raise Exception("input channel can't match output channel")
        
    #第一个分支：经过两个卷积层
    #第二个分支：恒等变换，如果需要为了使size相同，原始输入需要降采样，这里的降采样采用maxpooling
    
    #conv1可能会降采样：如果之前strides被设置成了(2,2)，就会降采样
    pre1 = tf.layers.conv2d(x, 
                             output_channel,
                             (3,3), #卷积核大小
                             strides = strides, #如果是(2,2)，那么就实现了缩小的功能
                             padding = 'same',
                             activation = None,
                             name = 'conv1')
    bn = tf.layers.batch_normalization(pre1, training = is_training)
    conv1 = tf.nn.relu(bn)
    
    #conv2不会降采样，因为conv1已经降采样过了
    pre2 = tf.layers.conv2d(conv1, 
                             output_channel,
                             (3,3), #卷积核大小
                             strides = (1, 1), #conv2不会降采样
                             padding = 'same',
                             activation = None,
                             name = 'conv2')
    bn2 = tf.layers.batch_normalization(pre2, training = is_training)
    conv2 = tf.nn.relu(bn2)
    
    #分支加法：
    #1. 如果conv1进行了降采样，原始输入也要降采样
    if increase_dim:
        #降采样，让我们的输出的大小缩小
        pooled_x = tf.layers.average_pooling2d(x,
                                               (2,2), #pooling核的大小一般是(2,2)
                                               (2,2), #步长一般也是(2,2)
                                               padding = 'valid') #因为我们的cifar10的图像大小是32*32,因为步长是(2,2),所以这里写valid和same都一样
                                                                  #如果图像是33*33,步长是(2,2),那么valid会舍去边角料，从而丢失信息，我们有时候不想丢失信息
        #注意，此时我们原始输入的通道数 我们需要的output_channel的一半
        #为了和之后的conv2相加，我们需要将pooled_x再添加通道数
        #tf.pad()第二个参数：[None, img_width, img_height, channel]
        #变成 [,,, channel * 2]
        padded_x = tf.pad(pooled_x,
                           [[0,0],
                            [0,0], #左侧，右侧
                            [0,0], #上侧，下侧
                            [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    
    #最后两个分支相加：
    output_x = conv2 + padded_x
    
    return output_x
    

### ResNet实现

In [6]:
#num_residual_block:每一层有多少残差连接块
#num_subsampling:降采样次数
#num_filter_base:最初的通道数目，（注意降采样的时候，通道数目*2。或者说通道数目加倍的时候，我们需要降采样）
#class_num:为了适应多种类别数目不同的数据集
def res_net(x,
            num_residual_blocks,
            num_filter_base,
            class_num,
            is_training):
    """residual network implementation"""
    """
    Args:
    - x:
    - num_residual_blocks: eg:[3,4,6,3]
    - class_num:
    """
    
    num_subsampling = len(num_residual_blocks)
    layers = []
    # x: [None, width, height, channel]
    # x[input_size]: [width, height, channel], 将第一个维度去掉
    input_size = x.get_shape().as_list()[1:]
    
    #先让输入层进入一个普通的CN
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x,
                                 num_filter_base, #输出通道数 == 输入通道数，不变
                                 (3,3),
                                 strides = (1,1),
                                 padding = "same", 
                                 activation = tf.nn.relu,
                                 name = 'conv0')
        layers.append(conv0)
        
    #因为此例中，cifar10的维度小只有32*32，所以这一层我们就不用ResNet的maxpooling了
    #开始残差块实现：
    #eg. num_subsampling = 4
    #2 ** sample_id = [1, 2, 4, 8]
    #所以每一个残差块的输出通道就变成了 原始输入通道数 的1,2,4,8倍
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                        layers[-1],
                        num_filter_base * (2 ** sample_id),
                        is_training)
                layers.append(conv)
    
    #验证我们每一层是否是我们需要的：
    #验证：宽高是否缩小，通道数是否加倍.
    #layers[-1].get_shape().as_list[1:] 就是[width, height, channel]
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
            == [input_size[0] / multiplier,
                input_size[1] / multiplier,
                num_filter_base * multiplier]
    
    #最后是 avg_pooling, 全连接full connection
    with tf.variable_scope('fc'):
        #avg_pooling就直接用reduce_mean实现平均，注意我们只对第2，3维度平均
        #因为layers[-1]，也就是最后一层的conv的维度有四个：[None, width, height, channel]
        #注意，经过global pooling之后就变成了一个值，也就是32*32变成了1个值,也就是32*32个像素的平均值
        #因为global pooling的kernel size = img_width, img_height
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        
        #fc:因为最后要传入给softmax(), softmax()需要的参数叫logits
        logits = tf.layers.dense(global_pool, class_num) #分类成10类，所以class_num会是10
        layers.append(logits)
        
    return layers[-1]
        

In [7]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1,3,32,32])
x_image = tf.transpose(x_image, perm = [0,2,3,1])

In [8]:
is_training = tf.placeholder(tf.bool, [])

In [9]:
#随机设residual_block, 我们的cifar10的通道数是32？类别是10
y_ = res_net(x_image, [2,2,2,2], 64, 10, is_training)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [10]:
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

In [11]:
#bool:
#predict = p_y_1 > 0.5

#indexes(int):
#第一个维度是同一类别下每个样本，第二维度是同一个样本下的每个类别（一个类别有一个概率值）
predict_indexes = tf.argmax(y_, 1) #1代表第二个维度，求一个样本，哪个类别的概率值最大

#int64:
#predict_float = tf.cast(predict, tf.int64)

#bool:
correct_prediction = tf.equal(predict_indexes, y) #两个int64的array比较

#float64:
correct_prediction_float = tf.cast(correct_prediction, tf.float32)

#float64, 因为correct_prediction_float是[1.0, 0.0, 1.0, ...] 而reduce_mean就是看有多少1，然后除以len
accuracy = tf.reduce_mean(correct_prediction_float)

### AdamOptimizer
AdamOptimizer(learning_rate)：梯度下降的变种，是反向传播算法的变种，初始learning_rate是0.01，目标最小化loss

In [12]:
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

### cifar10 数据处理
因为mini-batch，所以需要在cifar10上循环遍历数据，每次在sess.run()的feed_dict塞入不同数据

In [13]:
#filenames: 对 训练数据集 和 测试数据集 是分开的
#need_shuffle: 训练数据集需要shuffle，测试数据集不需要shuffle
class CifarData:
    def __init__(self, filenames, need_shuffle):
        #读入数据：
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            #data的对应位置的图像的label，也是label对应位置的label，所以把它们两个zip起来
            all_data.append(data)
            all_labels.append(labels)
            #不再是只调取0，1类 
#             for item, label in zip(data, labels):
#                 if label in [0,1]:
#                     all_data.append(item)
#                     all_labels.append(label)
        #data是numpy的矩阵，
        #item是numpy的向量
        #all_data里面包含了许多numpy的向量，现在我们将这些向量纵向合并到一起，成为一个矩阵：用np.vstack()
        #all_labels里面包含了许多label的向量，现在我们将这些向量横向合并到一起，成为一个矩阵（但是这个矩阵只有一行：n * 1）：用np.hstack()
        self._data = np.vstack(all_data) 
        self._data = self._data / 127.5 - 1 #最后做的改进：将0-255的值二分成0，1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0] #多少个样本
        self._need_shuffle = need_shuffle
        self._indicator = 0 #遍历到数据集的哪个位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self): #在训练集上shuffle
        # np.random.permutation(6) -> [3,2,5,1,4,0]
        p = np.random.permutation(self._num_examples) #得到一个混的全排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size): #每次返回batch_size个数个样本
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() #就算遍历完，可是如果可以shuffle，我们的数据可以复用
                self._indicator = 0
                end_indicator = batch_size + 0
            else:
                raise Exception("have no more examples") #已经遍历完了并且不能shuffle
        if end_indicator > self._num_examples:
            #此时说明我们输入的batch_size比我们的所有样本都要大
            raise Exception("batch size is larger than example size")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

### 测试我们的CifarData
1. 首先cifar10拥有50k个样本（10个类别），我们现在是二分类问题，只选取label为0，1的
2. 所以应该有 50k * (1/5) = 10k个样本 (10000, 3072)

In [14]:
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch') ]

train_data = CifarData(train_filenames, True)

In [15]:
test_data = CifarData(test_filenames, False)

In [16]:
batch_data, batch_labels = train_data.next_batch(3)

### 执行流程图

In [ ]:
init = tf.global_variables_initializer() #执行初始化
#开启Session，相当于打开绘画
#sess.run()执行计算图
#1. 计算目标:
#表示要训练：[loss, accuracy, train_op]
#表示要测试：[loss, accuracy]

#2. feed_dict:塞入数据
# x: x要塞的数据
# 因为mini-batch，所以需要在cifar10上循环遍历数据，每次塞入不同数据

batch_size = 20 #每次取20张图来 训练 或者 测试
train_steps = 100000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        #因为train_op返回的东西没有意义，所以用 _ 下划线接住返回值
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict = {
                x: batch_data, 
                y: batch_labels,
                is_training: True})  #注意格式！
        #===============================
        #======训练=====================
        if (i+1) % 1000 == 0: #使用(i+1)就可以输出500和1000
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f"\
                 % (i+1, loss_val, accu_val))
        #===============================
        #======测试=====================
        if (i+1) % 5000 == 0: #使用(i+1)就可以输出500和1000
            test_data = CifarData(test_filenames, False)
            all_test_acc_vals = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels,
                        is_training: False
                    })
                all_test_acc_vals.append(test_acc_val)
            test_acc = np.mean(all_test_acc_vals)
            print("[Test] Step: %d, acc: %4.5f" % (i+1, test_acc))